In [10]:
# Load the modules we need
import os
from bs4 import BeautifulSoup
import json
import re
from datetime import datetime


# FILE being examined
the_file = "9_11_healing_the_mothers_who_found_forgiveness_friendship.html"

# Everything else we need to open the file
dir_path = "/Users/john/Code/tedmetadata/html/"
the_doc = open(os.path.join(dir_path, the_file),"r") # OR: the_doc = open(dir_path+the_file,"r")

In [11]:
text = the_doc.read()
soup = BeautifulSoup(text, "html5lib")
my_list = [i.string.lstrip('q("talkPage.init", {\n\t"el": "[data-talk-page]",\n\t "__INITIAL_DATA__":')
           .rstrip('})')
           for i in soup.select('script') 
           if i.string and i.string.startswith('q')]

In [12]:
# Read first layer of JSON and get out those elements we want
pre_json = '{"' + "".join(my_list)

In [13]:
my_json = json.loads(pre_json)

NO ERROR! Why is this returning an error when it's a function but not returning an error on its own?

In [14]:
slug = my_json['slug']
vcount = my_json['viewed_count']
event = my_json['event']

# Read second layer of JSON and get out listed elements:
properties = "filmed,published" # No spaces between terms!
talks_listed = str(my_json['talks']).split(",")
regex_list = [".*("+i+").*" for i in properties.split(",")]
matches = []
for e in regex_list:
    filtered = filter(re.compile(e).match, talks_listed)
    indexed = "".join(filtered).split(":")[1]
    matches.append(indexed)
filmed = datetime.utcfromtimestamp(float(matches[0])).strftime('%Y-%m-%d')

#published = datetime.utcfromtimestamp(float(matches[1])).strftime('%Y-%m-%d')
#return slug, vcount, event, filmed, #published

ValueError: could not convert string to float: ' 1304362260}]'

In [ ]:
matches[1]